In [1]:
# import the libraries
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import polars as pl

import xgboost as xgb

In [2]:
# read the dataset
data = pl.read_csv("../data/new_pullreq.csv").to_pandas()
data.head()

,id,project_id,github_id,pull_request_id,ownername,reponame,merged_or_not,lifetime_minutes,mergetime_minutes,num_commits,...,doc_files_open,other_files_open,src_churn_open,test_churn_open,code_churn_open,churn_addition_open,churn_deletion_open,code_chunk_num_open,commits_on_files_touched_close,test_inclusion_open
0,1,13708387,1895,16946031,stylelint,stylelint,1,237,237.0,1,...,0,1,0,0,0,0,0,1,79,0
1,2,100297899,353,51228565,Joaogarciadelima,checklistos,0,1410,NaN,1,...,0,0,0,0,0,0,0,1,38,0
2,3,93139005,404,42975776,binary-com,SmartCharts,1,4,4.0,1,...,0,1,0,0,0,0,0,1,175,0
3,4,15059440,3434,34700062,letsencrypt,boulder,1,52,52.0,1,...,0,0,9,104,113,110,3,2,24,1
4,5,29684214,486,34175163,PyCQA,astroid,1,2414,2414.0,1,...,0,0,33,27,60,60,0,2,7,1


In [3]:
# drop the unwanted columns
for i in data.iloc[:5, :][[ 'ownername', 'reponame', 'github_id']].values:
    print(i)

['stylelint' 'stylelint' 1895]
['Joaogarciadelima' 'checklistos' 353]
['binary-com' 'SmartCharts' 404]
['letsencrypt' 'boulder' 3434]
['PyCQA' 'astroid' 486]


In [4]:
# remove accepted requests and remove data where gender is unknown
rejected_data = data.loc[data['merged_or_not'] == 0]
rejected_data = rejected_data.loc[rejected_data['contrib_gender'].notna()]

In [5]:
rejected_data.shape

(364116, 140)

In [6]:
# drop columns that we do not need
rejected_data = rejected_data.drop(['ownername', 'reponame', 'id', 'project_id', 'github_id', 'creator_id'], axis=1)

In [7]:
# characteristics are from the paper (https://yuyue.github.io/res/paper/newPR_MSR2020.pdf)
# prepare characteristics to drop (contributor characteristics, project characteristics, are characteristics that the contributor cannot control)
contributor_characteristics = ['acc_commit_num', 'first_pr', 'core_member', 'same_country', 'same_affiliation', 'contrib_open', 'contrib_cons', 'contrib_extra', 'contrib_agree', 'contrib_neur', 'inte_open', 'inte_cons', 'inte_extra', 'inte_neur', 'inte_agree', 'open_diff', 'cons_diff', 'extra_diff', 'agree_diff', 'neur_diff', 'social_strength', 'account_creation_days', 'prior_review_num', 'first_response_time', 'contrib_country', 'inte_country', 'prior_interaction', 'contrib_affiliation', 'inte_affiliation', 'contrib_first_emo', 'inte_first_emo', 'contrib_follow_integrator']
project_characteristics = ['language', 'project_age', 'pushed_delta', 'pr_succ_rate', 'open_issue_num', 'open_pr_num', 'fork_num']
unable_to_control = ['followers', 'part_num_issue', 'part_num_commit', 'part_num_pr', 'pr_comment_num', 'num_issue_comments', 'has_comments', 'has_participants', 'inte_comment', 'has_exchange', 'num_comments_con', 'first_close_minutes', 'num_issue_comments', 'num_participants', 'lifetime_minutes', 'ci_exists',  'reviewer_comment', 'contrib_comment', 'contrib_rate_author'] # features that contributor cannot control

In [8]:
# drop unwanted characteristics
rejected_data.drop(contributor_characteristics, axis=1, inplace=True)
rejected_data.drop(project_characteristics, axis=1, inplace=True)
rejected_data.drop(unable_to_control, axis=1, inplace=True)

In [9]:
rejected_data.shape

(364116, 77)

In [ ]:
# label encode the contrib_gender column to 0 and 1
le = LabelEncoder()
le.fit(rejected_data['contrib_gender'])
rejected_data['contrib_gender'] = le.transform(rejected_data['contrib_gender'])

In [ ]:
# encode all the columns that are not numerical, and save their label encoders
encoders = { 'contrib_gender' : le }
for column in rejected_data.select_dtypes(include=['object']).columns:
    le_col = LabelEncoder()
    le_col.fit(rejected_data[column])
    rejected_data[column] = le_col.transform(rejected_data[column])
    encoders[column] = le_col

In [ ]:
# find correlation to see which features are important
correlation_data = rejected_data.drop(['merged_or_not'], axis=1)
corr = correlation_data.corr()
correlation = corr['contrib_gender']
sorted(correlation.items(), key=lambda x: -x[1])

In [ ]:
# chose features that have correlation greater than 0.02
threshold = 0.02
high_correlation = list(filter(lambda x : x[1] >= threshold, sorted(correlation.items(), key=lambda x: -x[1])))
high_correlation_feat = list(map(lambda x : x[0], high_correlation))
high_correlation_feat.remove('contrib_gender')

In [ ]:
# get data and clean up data by removing n/a values
features = high_correlation_feat
print(features)
# features = ['account_creation_days',  'contrib_open',  'part_num_issue',  'perc_neu_emotion',  'num_participants',  'perc_pos_emotion',  'ci_test_passed',  'asserts_per_kloc',  'followers',  'num_issue_comments',  'perc_neg_emotion',  'lifetime_minutes',  'test_inclusion',  'ci_exists',  'acc_commit_num',  'core_member',  'num_comments',  'at_tag',  'pushed_delta',  'part_num_commit',  'first_response_time',  'ci_latency',  'project_age',  'ci_build_num',  'first_pr',  'hash_tag',  'test_churn',  'num_commit_comments',  'comment_conflict',  'prev_pullreqs',  'bug_fix',  'churn_deletion',  'has_comments',  'has_participants',  'reviewer_comment',  'contrib_comment',  'inte_comment',  'has_exchange',  'same_country',  'perc_contrib_neu_emo',  'social_strength',  'agree_diff',  'contrib_rate_author',  'inte_open',  'contrib_follow_integrator',  'extra_diff',  'num_comments_con',  'first_close_minutes',  'perc_inte_pos_emo',  'neur_diff',  'perc_contrib_neg_emo',  'perc_inte_neu_emo',  'cons_diff',  'same_affiliation',  'test_inclusion_open',  'perc_inte_neg_emo',  'num_code_comments_con',  'friday_effect',  'test_churn_open',  'other_comment',  'perc_contrib_pos_emo',  'integrator_availability',  'churn_deletion_open']
rejected_data_cleaned = rejected_data[[*features, 'contrib_gender']].dropna()
X = rejected_data_cleaned[features]
y = rejected_data_cleaned['contrib_gender']

In [ ]:
rejected_data_cleaned.shape

In [ ]:
# split into train test val split
train_ratio = 0.70
test_ratio = 0.20
val_ratio = 0.10

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1-train_ratio)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(val_ratio+test_ratio))

In [ ]:
# train the random forest classifier model on the training data
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)